In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from utils.dataset_helper import load_imdb
from utils.utils import vectorize_keywords_docs

In [4]:
X_train_corpus, y_train, X_test_corpus, y_test = load_imdb('dataset/aclImdb/')

In [38]:
X_train, X_test, cv = vectorize_keywords_docs(X_train_corpus, X_test_corpus, return_cv=True)

In [21]:
X_train = X_train['docs']
X_test = X_test['docs']

In [31]:
clf = LogisticRegression(solver='lbfgs', max_iter=500)
clf.fit(X_train, y_train)

In [52]:
weights = clf.coef_[0]
words = cv.get_feature_names()
indices = np.argsort(weights)

In [91]:
for i in indices[::-1][:100]:
    print (f'{words[i]:<20} {weights[i]:>15}')

7/10                 4.336600803854769
8/10                 2.4144803343466053
refreshing           2.149615732689694
hooked               1.9160959423876192
superbly             1.7894879379334536
noir                 1.7296280028657474
haunting             1.7237652408592257
appreciated          1.6910950362008819
rare                 1.6036769678259306
incredible           1.5987104059769555
9/10                 1.5451496998137006
10/10                1.5358242170640997
captures             1.5202431318983916
7                    1.5105610563975609
expressions          1.4887995876571851
wonderfully          1.4858698721963524
existed              1.4539629189775287
worlds               1.4277266219290567
prince               1.4256438134203013
steals               1.4038803996103169
gem                  1.3759687759165928
hong                 1.3732693826174451
definite             1.3631659379077696
flawless             1.3621792061142197
batman               1.3618029295862344
tr

In [92]:
for i in indices[:100]:
    print (f'{words[i]:<20} {weights[i]:>15}')

4/10                 -4.012366115209716
3/10                 -3.251198633312059
2/10                 -2.9386533087272246
1/10                 -2.8169183564608957
waste                -2.2803912241003204
disappointment       -2.2438871650831156
poorly               -2.1651501966323674
worst                -2.145820482480448
unwatchable          -2.066573374019713
uninteresting        -1.9493112878368544
stinker              -1.9386152854525236
unfunny              -1.9374091554878277
obnoxious            -1.9084168113030375
boredom              -1.8802390169997014
forgettable          -1.850572322951797
laughable            -1.8045332272183732
mildly               -1.7829370896084318
cardboard            -1.776961515005667
pointless            -1.7490694226446606
stupidity            -1.7148412661292454
wooden               -1.7138650664490243
awful                -1.6733317618380896
uninspired           -1.5845466813782914
lousy                -1.548588199595469
dull                 -1